In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
#from konlpy.tag import mecab
from gensim.models import Word2Vec
from scipy.spatial import distance
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer

## Data preprocessing

In [3]:
raw_df = pd.read_csv('theme_contents.csv', encoding = 'utf-8', index_col=0)
raw_df.dropna(inplace=True)

In [4]:
raw_df.head()

,theme,contents
0,2차전지,한국무역보험공사(K-SURE)는 27일 이인호 무역보험공사 사장이 신종 코로나바이러...
1,2차전지,코스피 상장사 이엔플러스가 2차전지 효율을 극대화할 수 있는 '액상 그래핀' 개발...
2,2차전지,이엔플러스(074610)가 2차전지 효율을 극대화할 수 있는 액상 그래핀을 개발해...
3,2차전지,[[특징주]]이엔플러스가 세계 최초로 2차전지 효율을 극대화할 수 있는 액상 그래...
4,2차전지,2차전지 성장세로 관련 기업들이 함박웃음을 짓고 있다. 국내 배터리 3사의 연이은...


In [5]:
# only leave korean chars

raw_df = raw_df[['theme', 'contents']]
raw_df['contents'] = raw_df['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
raw_df.head()

,theme,contents
0,2차전지,한국무역보험공사는 일 이인호 무역보험공사 사장이 신종 코로나바이러스 감염증코로나 사...
1,2차전지,코스피 상장사 이엔플러스가 차전지 효율을 극대화할 수 있는 액상 그래핀 개발에 성...
2,2차전지,이엔플러스가 차전지 효율을 극대화할 수 있는 액상 그래핀을 개발해 오는 월부터 본...
3,2차전지,특징주이엔플러스가 세계 최초로 차전지 효율을 극대화할 수 있는 액상 그래핀을 개발...
4,2차전지,차전지 성장세로 관련 기업들이 함박웃음을 짓고 있다 국내 배터리 사의 연이은 증설...


In [8]:
# tokenize data using kobert tokenizer

tok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)

raw_tokenized_data = []
for content in tqdm(raw_df['contents']):
    token = sp(content)
    raw_tokenized_data.append(token)

using cached model


100%|██████████| 26700/26700 [00:14<00:00, 1904.92it/s]


In [9]:
# make train_data

train_data = raw_df.copy()
train_data['tokens'] = pd.Series(raw_tokenized_data)

In [10]:
train_data.head()

,theme,contents,tokens
0,2차전지,한국무역보험공사는 일 이인호 무역보험공사 사장이 신종 코로나바이러스 감염증코로나 사...,"[▁한국, 무역, 보험, 공사, 는, ▁일, ▁이, 인, 호, ▁무역, 보험, 공사..."
1,2차전지,코스피 상장사 이엔플러스가 차전지 효율을 극대화할 수 있는 액상 그래핀 개발에 성...,"[▁코스피, ▁상장, 사, ▁이, 엔, 플러스, 가, ▁, 차전, 지, ▁, 효율,..."
2,2차전지,이엔플러스가 차전지 효율을 극대화할 수 있는 액상 그래핀을 개발해 오는 월부터 본...,"[▁이, 엔, 플러스, 가, ▁, 차전, 지, ▁, 효율, 을, ▁극대화, 할, ▁..."
3,2차전지,특징주이엔플러스가 세계 최초로 차전지 효율을 극대화할 수 있는 액상 그래핀을 개발...,"[▁, 특징주, 이, 엔, 플러스, 가, ▁세계, ▁최초로, ▁, 차전, 지, ▁,..."
4,2차전지,차전지 성장세로 관련 기업들이 함박웃음을 짓고 있다 국내 배터리 사의 연이은 증설...,"[▁, 차전, 지, ▁성장세, 로, ▁관련, ▁기업들, 이, ▁함, 박, 웃음, 을..."


## Model training

In [11]:
news_tokens = train_data['tokens'].tolist()  # data input as list

# params
v_dimension = 300
v_window = 8

model = Word2Vec(sentences = news_tokens, size = v_dimension, window = v_window, min_count = 5, workers = 4, sg = 0)

In [12]:
model.wv.vectors.shape

(7212, 300)

In [14]:
# save model
model.save("word2vec.model")

## Get theme vector representations

In [15]:
# get sentence vectors without normalization

def without_normal(tokens):
    vectors = []
    for token in tokens:
        init_v = np.array([0.0]*v_dimension)
        for word in token:
            word_vectors = model.wv
            if word in word_vectors.vocab:
                v = model.wv[word]
                init_v = init_v + v
        vectors.append(init_v)
    
    frame = { 'themes': train_data['theme'].tolist(), 'vectors': vectors }
    result = pd.DataFrame(frame) 
    
    return result

In [16]:
# get theme vectors without normalization

def theme_without_normal(news_df):
    theme_list = []
    vector_list = []
    for theme in news_df['themes'].unique():
        temp_df = news_df.loc[news_df['themes'] == theme]
        add_v = np.array([0.0]*v_dimension)
        for vec in temp_df['vectors']:
            add_v  = add_v + vec
        theme_list.append(theme)
        vector_list.append(add_v)
        
    frame = { 'themes': theme_list, 'vectors': vector_list }
    result = pd.DataFrame(frame)
    
    return result

In [17]:
news_vectors_df = without_normal(train_data['tokens'])

In [18]:
news_vectors_df.head()

,themes,vectors
0,2차전지,"[-29.128378542140126, 27.273291456745937, -8.1..."
1,2차전지,"[-41.46250488201622, -30.55584678146988, 191.7..."
2,2차전지,"[9.841240237117745, 22.945401730015874, 87.265..."
3,2차전지,"[-20.97598976327572, -6.953442683443427, 5.432..."
4,2차전지,"[-33.22875138127711, 15.341459663584828, 135.8..."


In [19]:
theme_vectors_df = theme_without_normal(news_vectors_df)

In [20]:
theme_vectors_df.head()

,themes,vectors
0,2차전지,"[-5728.885039504268, -1713.120836096663, 2425...."
1,3D프린터,"[-1601.962142577162, -5690.222019276283, 14515..."
2,4대강,"[-4492.6769370889815, 246.49643547526284, 3120..."
3,5G,"[-5324.962389420718, -2843.6464109625995, 8535..."
4,AR,"[4351.3103650522535, -178.3511861403931, 10230..."


In [21]:
# convert array to list for output file
theme_vectors_df['vectors'] = theme_vectors_df['vectors'].apply(lambda x: x.tolist())
theme_vectors_df.to_csv('theme_vectors_xnorm.csv')